# Voice translator with Azure AI

<img src="https://github.com/retkowsky/azure-ai-translator/blob/main/webapp3.jpg?raw=true">

In [1]:
import datetime
import pandas as pd
import gradio as gr
import openai
import os
import sys

from azure.ai.translation.text import TextTranslationClient
from azure.core.credentials import AzureKeyCredential
from azure.ai.translation.text.models import InputTextItem
from dotenv import load_dotenv
from IPython.display import Audio
from openai import AzureOpenAI

In [2]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [3]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 05-Mar-2025 11:21:53


## 1. Settings

In [4]:
def check_openai_version():
    """
    Check OpenAI version
    """
    try:
        installed_version = openai.__version__
        version_number = float(installed_version.split('.')[0])

    except (AttributeError, ValueError):
        print("Invalid OpenAI version format")
        return

    print(f"Installed OpenAI version: {installed_version}\n")

    if version_number < 1.0:
        print("[Warning] You should upgrade OpenAI to have version >= 1.0.0")
        print("To upgrade, run: %pip install openai --upgrade")

    else:
        print(f"[OK] OpenAI version {installed_version} is greater than version 1.0.0")

In [5]:
check_openai_version()

Installed OpenAI version: 1.61.1

[OK] OpenAI version 1.61.1 is greater than version 1.0.0


In [6]:
load_dotenv("azure.env")

# Azure AI Translator
azure_ai_translator_key = os.getenv("AZURE_AI_TRANSLATION_KEY")
azure_ai_translator_endpoint = os.getenv("AZURE_AI_TRANSLATION_ENDPOINT")
azure_ai_translator_region = os.getenv("AZURE_AI_TRANSLATION_REGION")

# Azure OpenAI Whisper model
openai.api_base = os.getenv("AOAI_ENDPOINT")
openai.api_key = os.getenv("AOAI_KEY")
openai.api_version = os.getenv("AOAI_VERSION")
openai.api_type = os.getenv("AOAI_TYPE")
deployment_id = os.getenv("AOAI_DEPLOYMENT")  # Whisper deployed name model
model = os.getenv("AOAI_MODEL")  # Whisper model

In [7]:
# Azure OpenAI Whisper client
aoai_client = AzureOpenAI(
    api_key=openai.api_key,
    api_version=openai.api_version,
    azure_endpoint=openai.api_base,
)

## 2. Speech to text with Azure OpenAI Whisper

In [8]:
def speech_to_text_whisper(audio_filename):
    """
    Transcribes speech from an audio file using Azure Speech to Text service.
    Input: audio file
    Output: text transcript
    """
    try:
        with open(audio_filename, "rb") as audio_file:
            result = aoai_client.audio.transcriptions.create(
                file=audio_file,
                model=deployment_id
            )
            return result.text

    except Exception as e:
        print(f"[Error] Speech to text error: {e}")
        return None

In [9]:
audio_filename = "sample.wav"

In [10]:
Audio(audio_filename, autoplay=False)

### 2.1 Test

In [11]:
# Test
results = speech_to_text_whisper(audio_filename)
print(results)

Hello. How are you today?


### 2.2 Speech to Text webapp

In [12]:
title = "Speech to Text with Azure OpenAI Whisper"
image_url = "https://github.com/retkowsky/azure-ai-translator/blob/main/logotranslate.png?raw=true"
logo = "<center> <img src= {} width=200px></center>".format(image_url)

stt_webapp = gr.Interface(
    speech_to_text_whisper,
    gr.Audio(sources="microphone", type="filepath", label="Audio Recording"),
    outputs=gr.Text(label="Text"),
    title=title,
    description=logo,
)

In [13]:
stt_webapp.launch(share=True)

* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://a48e2fbacf962b5f73.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 3. Languages supported

In [14]:
credential = AzureKeyCredential(azure_ai_translator_key)

text_translator = TextTranslationClient(
    credential=credential,
    region=azure_ai_translator_region
)

In [15]:
def print_languages(label, languages):
        """
        Print supported languages
        """
        print("\033[1;31;34m")
        if languages is not None:
            print(f"Number of supported {label} languages = {len(languages)}\n")
            print(f"{label.capitalize()} languages:")

            for idx, (key, value) in enumerate(languages.items(), start=1):
                print(f"{idx:03}\t{key:10} {value.name} ({value.native_name})")

        else:
            print(f"No supported {label} languages.")

In [16]:
response = text_translator.get_supported_languages()

In [17]:
print_languages("translation", response.translation)


Number of supported translation languages = 135

Translation languages:
001	af         Afrikaans (Afrikaans)
002	am         Amharic (አማርኛ)
003	ar         Arabic (العربية)
004	as         Assamese (অসমীয়া)
005	az         Azerbaijani (Azərbaycan)
006	ba         Bashkir (Bashkir)
007	bg         Bulgarian (Български)
008	bho        Bhojpuri (भोजपुरी)
009	bn         Bangla (বাংলা)
010	bo         Tibetan (བོད་སྐད་)
011	brx        Bodo (बड़ो)
012	bs         Bosnian (Bosanski)
013	ca         Catalan (Català)
014	cs         Czech (Čeština)
015	cy         Welsh (Cymraeg)
016	da         Danish (Dansk)
017	de         German (Deutsch)
018	doi        Dogri (डोगरी)
019	dsb        Lower Sorbian (Dolnoserbšćina)
020	dv         Divehi (ދިވެހިބަސް)
021	el         Greek (Ελληνικά)
022	en         English (English)
023	es         Spanish (Español)
024	et         Estonian (Eesti)
025	eu         Basque (Euskara)
026	fa         Persian (فارسی)
027	fi         Finnish (Suomi)
028	fil        Filipino (Filipino)


In [18]:
print_languages("transliteration", response.transliteration)


Number of supported transliteration languages = 42

Transliteration languages:
001	ar         Arabic (العربية)
002	as         Assamese (অসমীয়া)
003	be         Belarusian (беларуская)
004	bg         Bulgarian (Български)
005	bn         Bangla (বাংলা)
006	brx        Bodo (बड़ो)
007	el         Greek (Ελληνικά)
008	fa         Persian (فارسی)
009	gom        Konkani (कोंकणी)
010	gu         Gujarati (ગુજરાતી)
011	he         Hebrew (עברית)
012	hi         Hindi (हिन्दी)
013	ja         Japanese (日本語)
014	kk         Kazakh (Қазақ Тілі)
015	kn         Kannada (ಕನ್ನಡ)
016	ko         Korean (한국어)
017	ks         Kashmiri (کٲشُر)
018	ky         Kyrgyz (Кыргызча)
019	mai        Maithili (मैथिली)
020	mk         Macedonian (Македонски)
021	ml         Malayalam (മലയാളം)
022	mn-Cyrl    Mongolian (Cyrillic) (Монгол)
023	mni        Manipuri (ꯃꯩꯇꯩꯂꯣꯟ)
024	mr         Marathi (मराठी)
025	ne         Nepali (नेपाली)
026	or         Odia (ଓଡ଼ିଆ)
027	pa         Punjabi (ਪੰਜਾਬੀ)
028	ru         Russian (Русский)
029

In [19]:
print_languages("dictionary", response.dictionary)


Number of supported dictionary languages = 50

Dictionary languages:
001	af         Afrikaans (Afrikaans)
002	ar         Arabic (العربية)
003	bg         Bulgarian (Български)
004	bn         Bangla (বাংলা)
005	bs         Bosnian (Bosanski)
006	ca         Catalan (Català)
007	cs         Czech (Čeština)
008	cy         Welsh (Cymraeg)
009	da         Danish (Dansk)
010	de         German (Deutsch)
011	el         Greek (Ελληνικά)
012	en         English (English)
013	es         Spanish (Español)
014	et         Estonian (Eesti)
015	fa         Persian (فارسی)
016	fi         Finnish (Suomi)
017	fr         French (Français)
018	he         Hebrew (עברית)
019	hi         Hindi (हिन्दी)
020	hr         Croatian (Hrvatski)
021	hu         Hungarian (Magyar)
022	id         Indonesian (Indonesia)
023	is         Icelandic (Íslenska)
024	it         Italian (Italiano)
025	ja         Japanese (日本語)
026	ko         Korean (한국어)
027	lt         Lithuanian (Lietuvių)
028	lv         Latvian (Latviešu)
029	ms       

In [20]:
# Loading the supported languages into a dataframe
langlist = []

if response.translation is not None:
    for key, value in response.translation.items():
        langlist.append(
            {
                "Language_Code": key,
                "Language_Name": value.name,
                "Native_Name": value.native_name,
            }
        )

df_languages = pd.DataFrame(langlist)
df_languages

,Language_Code,Language_Name,Native_Name
0,af,Afrikaans,Afrikaans
1,am,Amharic,አማርኛ
2,ar,Arabic,العربية
3,as,Assamese,অসমীয়া
4,az,Azerbaijani,Azərbaycan
...,...,...,...
130,yua,Yucatec Maya,Yucatec Maya
131,yue,Cantonese (Traditional),粵語 (繁體)
132,zh-Hans,Chinese Simplified,中文 (简体)
133,zh-Hant,Chinese Traditional,繁體中文 (繁體)


In [21]:
print("Azure AI Translator supported languages")
print("\033[1;31;34m")
print(df_languages.to_string())

Azure AI Translator supported languages

    Language_Code            Language_Name           Native_Name
0              af                Afrikaans             Afrikaans
1              am                  Amharic                  አማርኛ
2              ar                   Arabic               العربية
3              as                 Assamese               অসমীয়া
4              az              Azerbaijani            Azərbaycan
5              ba                  Bashkir               Bashkir
6              bg                Bulgarian             Български
7             bho                 Bhojpuri               भोजपुरी
8              bn                   Bangla                 বাংলা
9              bo                  Tibetan              བོད་སྐད་
10            brx                     Bodo                  बड़ो
11             bs                  Bosnian              Bosanski
12             ca                  Catalan                Català
13             cs                    Czech       

In [22]:
df_languages.to_excel("azure_ai_translator_languages.xlsx")

In [23]:
# Get dict from dataframe
language_dict = df_languages.set_index("Language_Code")["Language_Name"].to_dict()
# Full names list
language_full_names = list(language_dict.values())
# Sort
language_full_names.sort()
# Language codes list
language_codes = list(language_dict.keys())
# Sort
language_codes.sort()
# Creation of dict
reverse_language_names = {v: k for k, v in language_dict.items()}

In [24]:
print(language_codes)

['af', 'am', 'ar', 'as', 'az', 'ba', 'bg', 'bho', 'bn', 'bo', 'brx', 'bs', 'ca', 'cs', 'cy', 'da', 'de', 'doi', 'dsb', 'dv', 'el', 'en', 'es', 'et', 'eu', 'fa', 'fi', 'fil', 'fj', 'fo', 'fr', 'fr-CA', 'ga', 'gl', 'gom', 'gu', 'ha', 'he', 'hi', 'hne', 'hr', 'hsb', 'ht', 'hu', 'hy', 'id', 'ig', 'ikt', 'is', 'it', 'iu', 'iu-Latn', 'ja', 'ka', 'kk', 'km', 'kmr', 'kn', 'ko', 'ks', 'ku', 'ky', 'ln', 'lo', 'lt', 'lug', 'lv', 'lzh', 'mai', 'mg', 'mi', 'mk', 'ml', 'mn-Cyrl', 'mn-Mong', 'mni', 'mr', 'ms', 'mt', 'mww', 'my', 'nb', 'ne', 'nl', 'nso', 'nya', 'or', 'otq', 'pa', 'pl', 'prs', 'ps', 'pt', 'pt-PT', 'ro', 'ru', 'run', 'rw', 'sd', 'si', 'sk', 'sl', 'sm', 'sn', 'so', 'sq', 'sr-Cyrl', 'sr-Latn', 'st', 'sv', 'sw', 'ta', 'te', 'th', 'ti', 'tk', 'tlh-Latn', 'tlh-Piqd', 'tn', 'to', 'tr', 'tt', 'ty', 'ug', 'uk', 'ur', 'uz', 'vi', 'xh', 'yo', 'yua', 'yue', 'zh-Hans', 'zh-Hant', 'zu']


In [25]:
print(reverse_language_names)

{'Afrikaans': 'af', 'Amharic': 'am', 'Arabic': 'ar', 'Assamese': 'as', 'Azerbaijani': 'az', 'Bashkir': 'ba', 'Bulgarian': 'bg', 'Bhojpuri': 'bho', 'Bangla': 'bn', 'Tibetan': 'bo', 'Bodo': 'brx', 'Bosnian': 'bs', 'Catalan': 'ca', 'Czech': 'cs', 'Welsh': 'cy', 'Danish': 'da', 'German': 'de', 'Dogri': 'doi', 'Lower Sorbian': 'dsb', 'Divehi': 'dv', 'Greek': 'el', 'English': 'en', 'Spanish': 'es', 'Estonian': 'et', 'Basque': 'eu', 'Persian': 'fa', 'Finnish': 'fi', 'Filipino': 'fil', 'Fijian': 'fj', 'Faroese': 'fo', 'French': 'fr', 'French (Canada)': 'fr-CA', 'Irish': 'ga', 'Galician': 'gl', 'Konkani': 'gom', 'Gujarati': 'gu', 'Hausa': 'ha', 'Hebrew': 'he', 'Hindi': 'hi', 'Chhattisgarhi': 'hne', 'Croatian': 'hr', 'Upper Sorbian': 'hsb', 'Haitian Creole': 'ht', 'Hungarian': 'hu', 'Armenian': 'hy', 'Indonesian': 'id', 'Igbo': 'ig', 'Inuinnaqtun': 'ikt', 'Icelandic': 'is', 'Italian': 'it', 'Inuktitut': 'iu', 'Inuktitut (Latin)': 'iu-Latn', 'Japanese': 'ja', 'Georgian': 'ka', 'Kazakh': 'kk', 'Kh

In [26]:
print(f"Number of supported languages with Azure AI Translator = {len(reverse_language_names)}")

Number of supported languages with Azure AI Translator = 135


In [27]:
def get_language_code(language_name):
    """
    Function to get language code by language name
    Input: language full name
    Output: language code name
    """
    return reverse_language_names.get(language_name)

### 3.1 Test

In [28]:
# Test
print(get_language_code("English"))

en


## 4.1 Translation

In [30]:
def azure_ai_translator(mytext, source_lang, target_lang):
    """
    Translates text from one language to another using Azure AI Translator.
    
    Args:
        mytext (str): The text to be translated.
        source_lang (str): The full name of the source language.
        target_lang (str): The full name of the target language.
    
    Returns:
        str: The translated text, or None if an error occurs.
    """
    try:
        credential = AzureKeyCredential(azure_ai_translator_key)
        
        text_translator = TextTranslationClient(
            credential=credential, region=azure_ai_translator_region)
        input_text_elements = [mytext]

        # Get language codes
        source_lang_code = get_language_code(source_lang)
        target_lang_code = [get_language_code(target_lang)]
        
        # Response
        response = text_translator.translate(body=input_text_elements,
                                             to_language=target_lang_code)
        translation = response[0] if response else None

        if translation:
            detected_language = translation.detected_language
            if detected_language:
                print(f"Detected languages of the input text: {detected_language.language} with score = {detected_language.score}.")
            for translated_text in translation.translations:
                print(f"\nText to translate to: '{translated_text.to}'")
            return translated_text.text

    except HttpResponseError as exception:
        if exception.error is not None:
            print(f"Error Code: {exception.error.code}")
            print(f"Message: {exception.error.message}")

### 4.1 Test

In [31]:
# Quick Test
result = azure_ai_translator("Welcome to this presentation", "English", "German")
print(result)

Detected languages of the input text: en with score = 1.0.

Text to translate to: 'de'
Willkommen zu dieser Präsentation


### 5. STT and Translation

In [32]:
def stt_and_translation(audio_filename, source_lang, target_lang):
    """
    Speech to Text and translation using the STT transcript
    """
    try:
        # Speech to text result
        transcript = speech_to_text_whisper(audio_filename)
        # Translation of the Speech to text result
        translation = azure_ai_translator(transcript, source_lang, target_lang)
        
        return transcript, translation
    
    except Exception as e:
        print(f"[Error] Speech to text and translation error: {e}")
        
        return None, None

### 5.1 Test

In [33]:
# Test
transcript, translation = stt_and_translation(audio_filename, "English", "Spanish")

transcript, translation

Detected languages of the input text: en with score = 1.0.

Text to translate to: 'es'


('Hello. How are you today?', 'Hola. ¿Cómo te va?')

### 5.2 Speech to Text and Translation using a webapp

In [36]:
image_url = "https://github.com/retkowsky/azure-ai-translator/blob/main/logotranslate.png?raw=true"
logo = "<center> <img src= {} width=200px></center>".format(image_url)
title = "Vocal translator using Azure AI services"
theme = "JohnSmith9982/small_and_pretty"  # https://huggingface.co/spaces/gradio/theme-gallery

translator_webapp = gr.Interface(
    stt_and_translation,
    inputs=[
        gr.components.Audio(
            sources="microphone", 
            type="filepath",
            label="Audio recording"
        ),
        gr.components.Dropdown(
            label="Audio source language",
            value="French",
            choices=language_full_names
        ),
        gr.components.Dropdown(
            label="Text target language",
            value="English",
            choices=language_full_names
        ),
    ],
    outputs=[
        gr.Text(label="Source text"),
        gr.Text(label="Translated text"),
    ],
    cache_examples=False,
    title=title,
    description=logo,
    #theme=theme,
)

In [37]:
translator_webapp.launch(share=True)

* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://d4efac49c85d0bb66a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Detected languages of the input text: fr with score = 1.0.

Text to translate to: 'en'
